# COVID19-EU

Statistics and Forecasting for the Coronavirus disease (COVID-19) in the European Union.

## Source of Data

 Johns Hopkins CSSE Repository: [https://github.com/CSSEGISandData/COVID-19](https://github.com/CSSEGISandData/COVID-19).

## Source of EU country names

Official website of the European Union: [https://europa.eu/european-union/about-eu/countries_en](https://europa.eu/european-union/about-eu/countries_en)

### Load libraries

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
%matplotlib inline 

### Load Data from Johns Hopkins CSSE Repository

In [ ]:
confirmed_data = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')